# Test Docker-Compose Setup

In this notebook, we'll test the docker-compose setup. First, open a terminal, 
`cd` into the root directory of this repo and run the following command (c.f.
[README.md](../README.md) for more details):
```bash
# .env must be available at the root as shown in .env.example
docker compose up -d
```

You should see a series of logs indicating a successful run of your services.
If so, run the following cells to test your services.

This notebook is designed to be minimalist and as standalone as possible, so 
don't hesitate to re-use it and adjust it to your needs.

## Setup

### Imports

In [1]:
# import required libs
import arxiv
from rich import print
import qdrant_client
from langchain_community.embeddings import HuggingFaceHubEmbeddings
from langchain.schema.document import Document
from langchain.vectorstores import Qdrant
from langchain_community.llms import VLLMOpenAI
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, PromptTemplate
from langchain.chains import RetrievalQA

### Config

In [2]:
# set constants
QDRANT_URL = "http://localhost"
QDRANT_PORT = 6335
EMBED_API_URL = "http://localhost:8081"
GEN_API_URL = "http://localhost:8000/v1"

In [3]:
import dotenv
envvars = dotenv.dotenv_values()

## Connect to Microservices

In [4]:
# connect to embed_api
embed_model = HuggingFaceHubEmbeddings(model=EMBED_API_URL)

# test it !
text = "What is deep learning?"
query_result = embed_model.embed_query(text)
print(f"Embeddings from '{envvars['EMBED_MODEL_ID']}'")
query_result[:3]

/home/melion/miniconda3/envs/ai_news/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Embeddings from 'BAAI/bge-small-en-v1.5'

[-0.0076529826, -0.02522551, -0.024398882]

In [5]:
# connect to gen_api
llm = VLLMOpenAI(
    openai_api_key="EMPTY",
    openai_api_base=GEN_API_URL,
    model_name=envvars["GEN_MODEL_ID"],
    # model_kwargs={"stop": ["."]},
)

# test it !
answer = llm.predict("Write me a poem about Machine Learning.")
print(f"Generation from '{envvars['GEN_MODEL_ID']}'")
print(answer)

Generation from 'google/gemma-2b-it'

Machines, vast and grand, with minds of steel,
A symphony of algorithms, we can't fail.
Learning from data, patterns emerge,
Discovering truths, with each passing year.

From medical scans to marketing trends,
They weave insights, the tapestry extends.
With each iteration, they refine and adapt,
Unleashing potential, a digital bath.

But machines, though potent, hold a human face,
With biases and limitations, a note.
We must ensure fairness, a moral guide,
To prevent harm, to protect our tide.

So let's embrace this revolution in our time,
Where humans and machines, a harmonious chime.
With collaboration and understanding, we'll soar,
To build a future, where possibilities soar.

In [6]:
# connect to qdrant
client = qdrant_client.QdrantClient(
    url=QDRANT_URL,
    port=6333
    # api_key="<qdrant-api-key>", # For Qdrant Cloud, None for local instance
)

# test it
collections = client.get_collections()
for c in collections.collections:
    client.delete_collection(c.name)
    print(f"Deleted collection '{c.name}'")

Deleted collection 'arxiv_documents'

## Data Collection

In [7]:
# run a query through Arxiv API
search = arxiv.Search(
    query="ti:deep AND ti:learning",
    max_results=300,
    sort_by=arxiv.SortCriterion.Relevance,
)

# display some results
for result in list(search.results())[:2]:
    print(f"--- '{result.title}' [{result.published}] ---")
    print(result.summary, "\n")

/tmp/ipykernel_25981/2060595641.py:9: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  for result in list(search.results())[:2]:


--- 'Deep Learning' [2018-07-20 18:20:34+00:00] ---

Deep learning (DL) is a high dimensional data reduction technique for
constructing high-dimensional predictors in input-output models. DL is a form
of machine learning that uses hierarchical layers of latent features. In this
article, we review the state-of-the-art of deep learning from a modeling and
algorithmic perspective. We provide a list of successful areas of applications
in Artificial Intelligence (AI), Image Processing, Robotics and Automation.
Deep learning is predictive in its nature rather then inferential and can be
viewed as a black-box methodology for high-dimensional function estimation.

--- 'Deep, Deep Learning with BART' [2022-02-28 18:23:41+00:00] ---

Purpose: To develop a deep-learning-based image reconstruction framework for
reproducible research in MRI.
  Methods: The BART toolbox offers a rich set of implementations of calibration
and reconstruction algorithms for parallel imaging and compressed sensing. In
this work, BART was extended by a non-linear operator framework that provides
automatic differentiation to allow computation of gradients. Existing
MRI-specific operators of BART, such as the non-uniform fast Fourier transform,
are directly integrated into this framework and are complemented by common
building blocks used in neural networks. To evaluate the use of the framework
for advanced deep-learning-based reconstruction, two state-of-the-art unrolled
reconstruction networks, namely the Variational Network [1] and MoDL [2], were
implemented.
  Results: State-of-the-art deep image-reconstruction networks can be
constructed and trained using BART's gradient based optimization algorithms.
The BART implementation achieves a similar performance in terms of training
time and reconstruction quality compared to the original implementations based
on TensorFlow.
  Conclusion: By integrating non-linear operators and neural networks into
BART, we provide a general framework for deep-learning-based reconstruction in
MRI.

## Feed Vectorstore

In [8]:
# create Langchain documents from query results
docs = [
    Document(
        page_content=result.summary,
        metadata={
            "title": result.title,
            "publish_date": result.published,
        },
    )
    for i, result in enumerate(search.results())
]

/tmp/ipykernel_25981/2918398908.py:10: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  for i, result in enumerate(search.results())


In [9]:
# sore results in qdrant
qdrant = Qdrant.from_documents(
    docs, 
    embed_model, 
    url=QDRANT_URL, 
    # prefer_grpc=True, 
    collection_name="arxiv_documents",
    batch_size=16,
)

## QA using Qdrant & vLLM

In [10]:
# from langchain import hub
# prompt = hub.pull("rlm/rag-prompt", api_url="https://api.hub.langchain.com")

prompt = ChatPromptTemplate(
    input_variables=['context', 'question'], 
    messages=[
        HumanMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=['context', 'question'], 
                template="You are an assistant for question-answering tasks. " \
                    "Use the following pieces of retrieved context to answer " \
                    "the question. If you don't know the answer, just say "\
                    "that you don't know. Use three sentences maximum and "\
                    "keep the answer concise.\nQuestion: {question} "\
                    "\nContext: {context} \nAnswer:"
            )
        )
    ]
)

In [11]:
# declare Q&A chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm, 
    retriever=qdrant.as_retriever(
        search_type="similarity",
        search_kwargs={
            "k": 4,
            "filter": None,
        }
    ), 
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True,
)

In [12]:
# run Q&A chain
question = "Give examples of how Vision Transformers can be applied to real-life problems."
result = qa_chain({"query": question})

# display the answer
print(result["result"])

Vision transformers can be applied to various real-world problems related to image and video analysis, including 
face matching, print media monitoring, industrial quality control, music scanning, strategy game playing, and 
automated machine learning.

In [13]:
print(result)

{
    'query': 'Give examples of how Vision Transformers can be applied to real-life problems.',
    'result': ' Vision transformers can be applied to various real-world problems related to image and video 
analysis, including face matching, print media monitoring, industrial quality control, music scanning, strategy 
game playing, and automated machine learning.',
    'source_documents': [
        Document(
            page_content='Humans are generally good at learning abstract concepts about objects and\nscenes (e.g.\\
spatial orientation, relative sizes, etc.). Over the last years\nconvolutional neural networks have achieved almost
human performance in\nrecognizing concrete classes (i.e.\\ specific object categories). This paper\ntests the 
performance of a current CNN (GoogLeNet) on the task of\ndifferentiating between abstract classes which are 
trivially differentiable for\nhumans. We trained and tested the CNN on the two abstract classes of horizontal\nand 
vertical orientation and determined how well the network is able to\ntransfer the learned classes to other, 
previously unseen objects.',
            metadata={
                'publish_date': '2016-06-17T12:51:23Z',
                'title': 'Learning Abstract Classes using Deep Learning'
            }
        ),
        Document(
            page_content='Deep learning with neural networks is applied by an increasing number of\npeople outside 
of classic research environments, due to the vast success of the\nmethodology on a wide range of machine perception
tasks. While this interest is\nfueled by beautiful success stories, practical work in deep learning on novel\ntasks
without existing baselines remains challenging. This paper explores the\nspecific challenges arising in the realm 
of real world tasks, based on case\nstudies from research \\& development in conjunction with industry, and 
extracts\nlessons learned from them. It thus fills a gap between the publication of\nlatest algorithmic and 
methodical developments, and the usually omitted\nnitty-gritty of how to make them work. Specifically, we give 
insight into deep\nlearning projects on face matching, print media monitoring, industrial quality\ncontrol, music 
scanning, strategy game playing, and automated machine learning,\nthereby providing best practices for deep 
learning in practice.',
            metadata={'publish_date': '2018-07-13T07:22:45Z', 'title': 'Deep Learning in the Wild'}
        ),
        Document(
            page_content="This article investigates deep learning methodologies for single-modality\nclinical data 
analysis, as a crucial precursor to multi-modal medical research.\nBuilding on Guo JingYuan's work, the study 
refines clinical data processing\nthrough Compact Convolutional Transformer (CCT), Patch Up, and the 
innovative\nCamCenterLoss technique, establishing a foundation for future multimodal\ninvestigations. The proposed 
methodology demonstrates improved prediction\naccuracy and at tentiveness to critically ill patients compared to 
Guo\nJingYuan's ResNet and StageNet approaches. Novelty that using image-pretrained\nvision transformer backbone to
perform transfer learning time-series clinical\ndata.The study highlights the potential of CCT, Patch Up, and 
novel\nCamCenterLoss in processing single modality clinical data within deep learning\nframeworks, paving the way 
for future multimodal medical research and promoting\nprecision and personalized healthcare",
            metadata={'publish_date': '2024-03-06T00:36:05Z', 'title': 'Multi-modal Deep Learning'}
        ),
        Document(
            page_content='This book is the result of a seminar in which we reviewed multimodal\napproaches and 
attempted to create a solid overview of the field, starting with\nthe current state-of-the-art approaches in the 
two subfields of Deep Learning\nindividually. Further, modeling frameworks are discussed where one modality 
is\ntransformed into the other, as well as models in which